In [1]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

In [2]:
device = torch.device("cuda") 
print(f"Using device: {device}")

Using device: cuda


In [3]:
data = pd.read_csv('train_data.csv')

In [4]:
data.columns

Index(['Unnamed: 0', 'file_name', 'examination', 'indication', 'technique',
       'comparison', 'findings', 'impression', 'clean_filenames', 'Files',
       'P-Folder', 'p-Folder', 'path', 'image_path', 'split'],
      dtype='object')

In [5]:
# data.drop(columns = ['Unnamed: 0', 'file_name', 'examination', 'indication', 'technique',
#        'comparison', 'findings', 'impression', 'clean_filenames', 'Files',
#        'P-Folder', 'p-Folder', 'path', 'image_path', 'split']

In [6]:
data.head()

,Unnamed: 0,file_name,examination,indication,technique,comparison,findings,impression,clean_filenames,Files,P-Folder,p-Folder,path,image_path,split
0,0,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,478e39da-ea1ef26f-de4c7167-eb8a0af0-7de6fd2a,files/p10/p10000032/s50414267/02aa804e-bde0afd...,train
1,1,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,71924c95-a7042c53-10843bb7-3febb74d-b597c5f2,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,train
2,2,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,8fb87406-7667e3fe-eced1e60-2adbd40a-63558751,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,train
3,3,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,bcf8fa5a-f17ea507-cff20d4d-6eeb3c66-e52796d8,files/p10/p10000032/s53189527/e084de3b-be89b11...,train
4,4,s58348111.txt,NaN,Newly diagnosed rheumatoid arthritis. Evaluat...,NaN,No comparison available at the time of dictation.,pleural effusions. Normal size of the cardia...,NaN,s58348111,files,p19,p19561401,fb0732a9-c3d578e5-30f56374-71357f8e-84c9c972,files/p10/p10000032/s53911762/68b5c4b1-227d048...,train


In [7]:
def preprocess_text(text):
    if pd.isna(text):  # Handle missing values
        return ""
    # Basic text cleaning
    text = text.lower()  # Convert to lowercase
    text = text.replace('\n', ' ')  # Replace newlines with space
    text = text.replace('___', '')  # Remove placeholders
    text = text.strip()  # Remove leading/trailing whitespace
    return text

In [8]:
text_columns = ['examination', 'indication', 'technique', 'comparison', 'findings', 'impression']
for column in text_columns:
    data[column] = data[column].apply(preprocess_text)

In [9]:
data

,Unnamed: 0,file_name,examination,indication,technique,comparison,findings,impression,clean_filenames,Files,P-Folder,p-Folder,path,image_path,split
0,0,s51155740.txt,,"history: m with sob during rifaximin infusion,...",chest pa and lateral,none.,frontal and lateral chest radiographs demonstr...,elevation of the left hemidiaphragm with atele...,s51155740,files,p19,p19007931,478e39da-ea1ef26f-de4c7167-eb8a0af0-7de6fd2a,files/p10/p10000032/s50414267/02aa804e-bde0afd...,train
1,1,s51155740.txt,,"history: m with sob during rifaximin infusion,...",chest pa and lateral,none.,frontal and lateral chest radiographs demonstr...,elevation of the left hemidiaphragm with atele...,s51155740,files,p19,p19007931,71924c95-a7042c53-10843bb7-3febb74d-b597c5f2,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,train
2,2,s51155740.txt,,"history: m with sob during rifaximin infusion,...",chest pa and lateral,none.,frontal and lateral chest radiographs demonstr...,elevation of the left hemidiaphragm with atele...,s51155740,files,p19,p19007931,8fb87406-7667e3fe-eced1e60-2adbd40a-63558751,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,train
3,3,s51155740.txt,,"history: m with sob during rifaximin infusion,...",chest pa and lateral,none.,frontal and lateral chest radiographs demonstr...,elevation of the left hemidiaphragm with atele...,s51155740,files,p19,p19007931,bcf8fa5a-f17ea507-cff20d4d-6eeb3c66-e52796d8,files/p10/p10000032/s53189527/e084de3b-be89b11...,train
4,4,s58348111.txt,,newly diagnosed rheumatoid arthritis. evaluat...,,no comparison available at the time of dictation.,pleural effusions. normal size of the cardiac...,,s58348111,files,p19,p19561401,fb0732a9-c3d578e5-30f56374-71357f8e-84c9c972,files/p10/p10000032/s53911762/68b5c4b1-227d048...,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368955,377105,s52688542.txt,chest (portable ap)chest (portable ap)i,year old woman with ett // eval infiltrate,,"chest radiograph since , most recently",,mild pulmonary edema is new. severe consolida...,s52688542,files,p12,p12057859,9e49c4e2-518c7a0a-a7e30554-3b7f5f03-4f71e1e4,files/p19/p19999733/s57132437/428e2c18-5721d8f...,train
368956,377106,s59179690.txt,chest,f with ams // pneumonia,,,ap upright and lateral views of the chest prov...,cardiomegaly unchanged with mild pulmonary con...,s59179690,files,p12,p12057859,b71026b6-807374bf-47f0f0d7-4e984ae4-7f6aaaf3,files/p19/p19999733/s57132437/58c403aa-35ff8bd...,train
368957,377107,s59179690.txt,chest,f with ams // pneumonia,,,ap upright and lateral views of the chest prov...,cardiomegaly unchanged with mild pulmonary con...,s59179690,files,p12,p12057859,f677232c-4f48f565-3193bbb1-b1ee6f4b-37e7c6af,files/p19/p19999987/s55368167/58766883-376a15c...,train
368958,377108,s58463101.txt,chest (portable ap)chest (portable ap),"f p/w left hemiparesis, found to have b/l sah ...",,chest radiographs,,pulmonary and mediastinal vascular congestion ...,s58463101,files,p12,p12057859,ccfa71dc-e4d15466-609334a9-ce03f0aa-af08595c,files/p19/p19999987/s58621812/7ba273af-3d290f8...,train


In [10]:
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [11]:
def merge_text(row):
    # Merge columns with field labels for context
    merged_text = (
        f"Examination: {row['examination']} "
        f"Indication: {row['indication']} "
        f"Technique: {row['technique']} "
        f"Comparison: {row['comparison']} "
        f"Findings: {row['findings']} "
        f"Impression: {row['impression']}"
    )
    return merged_text.strip()


In [12]:
data['merged_text'] = data.apply(merge_text, axis=1)

In [13]:
# def generate_embedding(text, tokenizer, model, max_length=128):
#     inputs = tokenizer(
#         text,
#         max_length=max_length,
#         padding="max_length",
#         truncation=True,
#         return_tensors="pt"
#     )
#     with torch.no_grad():
#         outputs = model(**inputs)
#         cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
#     return cls_embedding.squeeze(0).cpu().numpy()


In [14]:
# # Generate embeddings for merged text
# merged_embeddings = []
# for text in tqdm(data['merged_text'], desc="Generating embeddings for merged text"):
#     embedding = generate_embedding(text, tokenizer, model)
#     merged_embeddings.append(embedding)

In [15]:
# # Attach embeddings back to the DataFrame
# sample_data_new['merged_embedding'] = merged_embeddings

# # Save processed DataFrame with embeddings
# output_path = "sample_data_with_merged_embeddings.pkl"
# sample_data_new.to_pickle(output_path)

In [16]:
# def generate_embedding(text, tokenizer, model, max_length=128):
#     """Generate embedding for a single piece of text."""
#     inputs = tokenizer(
#         text,
#         max_length=max_length,
#         padding="max_length",
#         truncation=True,
#         return_tensors="pt"
#     )
#     with torch.no_grad():
#         outputs = model(**inputs)
#         cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
#     return cls_embedding.squeeze(0).cpu().numpy()

# def process_embeddings_in_batches(data, tokenizer, model, batch_size=32, output_path="sample_data_with_merged_embeddings.pkl"):
#     """Process text data in batches, generating and saving embeddings."""

#     # Check if a partial file already exists
#     if os.path.exists(output_path):
#         with open(output_path, "rb") as f:
#             processed_data = pickle.load(f)
#             print(f"Loaded {len(processed_data)} already processed entries.")
#     else:
#         processed_data = {}

#     # Get the unprocessed indices
#     start_index = len(processed_data)
#     unprocessed_texts = data['merged_text'][start_index:]

#     print(f"Starting from index {start_index}, processing {len(unprocessed_texts)} entries.")

#     for i in tqdm(range(0, len(unprocessed_texts), batch_size), desc="Processing batches"):
#         batch_texts = unprocessed_texts[i:i + batch_size]
#         batch_indices = range(start_index + i, start_index + i + len(batch_texts))

#         # Generate embeddings for the current batch
#         batch_embeddings = [
#             generate_embedding(text, tokenizer, model)
#             for text in batch_texts
#         ]

#         # Update processed_data dictionary
#         for idx, embedding in zip(batch_indices, batch_embeddings):
#             processed_data[idx] = embedding

#         # Save progress to file
#         with open(output_path, "wb") as f:
#             pickle.dump(processed_data, f)

#         print(f"Saved up to index {batch_indices[-1]}.")

#     print("All embeddings processed and saved.")


# process_embeddings_in_batches(data, tokenizer, model, batch_size=32)


In [17]:
def generate_embedding(text, tokenizer, model, max_length=128):
    """Generate embedding for a single piece of text."""
    inputs = tokenizer(
        text,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
    return cls_embedding.squeeze(0).cpu().numpy()

def process_embeddings_in_batches(data, tokenizer, model, batch_size=64, output_path="train_data_text_embeddings.pkl"):
    """Process text data in batches, generating and saving embeddings."""

    # Check if a partial file already exists
    if os.path.exists(output_path):
        with open(output_path, "rb") as f:
            processed_data = pickle.load(f)
            print(f"Loaded {len(processed_data)} already processed entries.")
    else:
        processed_data = {}

    # Get the unprocessed indices
    start_index = len(processed_data)
    unprocessed_texts = data['merged_text'][start_index:]

    print(f"Starting from index {start_index}, processing {len(unprocessed_texts)} entries.")

    for i in tqdm(range(0, len(unprocessed_texts), batch_size), desc="Processing batches"):
        batch_texts = unprocessed_texts[i:i + batch_size]
        batch_indices = range(start_index + i, start_index + i + len(batch_texts))

        # Generate embeddings for the current batch
        inputs = tokenizer(
            list(batch_texts),
            max_length=128,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

        # Update processed_data dictionary
        for idx, embedding in zip(batch_indices, batch_embeddings):
            processed_data[idx] = embedding

        # Save progress to file
        with open(output_path, "wb") as f:
            pickle.dump(processed_data, f)

        print(f"Saved up to index {batch_indices[-1]}.")

    print("All embeddings processed and saved.")




In [18]:
import pickle

In [19]:
# process_embeddings_in_batches(data, tokenizer, model, batch_size=64)


In [20]:
import os
import pickle
import torch
from tqdm import tqdm


def generate_embedding(text, tokenizer, model, max_length=128):
    """
    Generate embedding for a single piece of text using a tokenizer and model.
    """
    inputs = tokenizer(
        text,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
    return cls_embedding.squeeze(0).cpu().numpy()


def process_embeddings_in_batches(data, tokenizer, model, batch_size=64, output_path="train_data_text_embeddings.pkl"):
    """
    Process text data in batches, generating and saving embeddings incrementally.

    Args:
        data (pd.DataFrame): A DataFrame containing text data with a 'merged_text' column.
        tokenizer: Tokenizer object for preprocessing text.
        model: Pretrained model for generating embeddings.
        batch_size (int): Number of texts to process in each batch.
        output_path (str): Path to save the generated embeddings.
    """

    # Load existing processed data if available
    processed_data = {}
    if os.path.exists(output_path) and os.path.getsize(output_path) > 0:  # Check for non-empty file
        with open(output_path, "rb") as f:
            try:
                processed_data = pickle.load(f)
                print(f"Loaded {len(processed_data)} already processed entries.")
            except EOFError:
                print("File is empty or corrupted. Starting fresh.")

    # Determine the start index for unprocessed texts
    start_index = len(processed_data)
    if 'merged_text' not in data.columns:
        raise KeyError("Input DataFrame must contain a 'merged_text' column.")
    unprocessed_texts = data['merged_text'][start_index:]

    if unprocessed_texts.empty:
        print("No new entries to process. Exiting.")
        return

    print(f"Starting from index {start_index}, processing {len(unprocessed_texts)} entries.")

    for i in tqdm(range(0, len(unprocessed_texts), batch_size), desc="Processing batches"):
        batch_texts = unprocessed_texts.iloc[i:i + batch_size].tolist()
        batch_indices = range(start_index + i, start_index + i + len(batch_texts))

        # Tokenize and process the batch
        inputs = tokenizer(
            batch_texts,
            max_length=128,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        ).to(model.device)  # Send inputs to the same device as the model

        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

        # Update processed data
        for idx, embedding in zip(batch_indices, batch_embeddings):
            processed_data[idx] = embedding

        # Save progress after each batch
        with open(output_path, "wb") as f:
            pickle.dump(processed_data, f)

        print(f"Saved embeddings up to index {batch_indices[-1]}.")

    print(f"All {len(processed_data)} embeddings processed and saved to {output_path}.")


In [21]:
process_embeddings_in_batches(data, tokenizer, model, batch_size=16)


Loaded 99264 already processed entries.
Starting from index 99264, processing 269696 entries.


Processing batches:   0%|          | 1/16856 [00:03<14:10:43,  3.03s/it]

Saved embeddings up to index 99279.


Processing batches:   0%|          | 2/16856 [00:05<11:43:19,  2.50s/it]

Saved embeddings up to index 99295.


...

Saved embeddings up to index 113967.


Processing batches:   5%|▌         | 920/16856 [31:37<9:49:25,  2.22s/it] 

Saved embeddings up to index 113983.


Processing batches:   5%|▌         | 920/16856 [31:38<9:08:04,  2.06s/it]


KeyboardInterrupt: 